# Modules for Datajoint

In [1]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")

import datajoint_utils as du
from importlib import reload

Connecting celiib@at-database.ad.bcm.edu:3306


In [2]:
import minfig
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons


In [3]:
minnie = minfig.configure_minnie(return_virtual_module=True)

# Old way of getting access to the virtual modules
# m65 = dj.create_virtual_module('minnie', 'microns_minnie65_02')

#New way of getting access to module
import datajoint as dj
from minfig import adapter_objects # included with wildcard imports
minnie = dj.create_virtual_module('minnie', 'microns_minnie65_02', add_objects=adapter_objects)

schema = dj.schema("microns_minnie65_02")
dj.config["enable_python_native_blobs"] = True
#(schema.jobs & "table_name='__baylor_segment_centroid_seg3'").delete()

INFO:datajoint.settings:Setting enable_python_native_blobs to True
INFO:datajoint.settings:Setting enable_python_native_blobs to True


In [ ]:
decimation_version = 0
decimation_ratio = 0.25

segments = (minnie.Mesh() & f'n_vertices  > {np.str(np.round(np.exp(12)).astype(np.int))}' & f'n_vertices  < {np.str(np.round(np.exp(15)).astype(np.int))}')
key_source =  minnie.Decimation.proj(decimation_version='version') & segments.proj() & "decimation_version=" + str(decimation_version)
key_source  

# The tables that should be used for pulling meshes

In [4]:
decimation_ratio = 0.25
decimation_version = 0

valid_segment_ids_with_nucleus_id = dj.U("segment_id") & (minnie.NucleusID() & "segment_id>0")
segments = (minnie.Mesh())# & f'n_vertices  > {np.str(np.round(np.exp(12)).astype(np.int))}' & f'n_vertices  < {np.str(np.round(np.exp(15)).astype(np.int))}')
key_source =  (minnie.Decimation.proj(decimation_version='version') 
            & segments.proj() 
            & f"decimation_ratio={decimation_ratio}" 
            & f"decimation_version={decimation_version}" 
            & valid_segment_ids_with_nucleus_id)
key_source

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses)
864691135635239593,0,0.25
864691135618175635,0,0.25
864691135584809548,0,0.25
864691135679304325,0,0.25
864691136143741847,0,0.25
864691136148842647,0,0.25
864691136143284631,0,0.25
864691136745551332,0,0.25
864691136315868311,0,0.25
864691136324529047,0,0.25


In [5]:
def nucleus_id_to_seg_id(nucleus_id):
    """
    Pseudocode:
    1) restrict the nucleus id table
    2) fetch for the segment id
    """
    nucl_key = dict(nucleus_id=nucleus_id)
    nucl_seg_id = (minnie.NucleusID()  & nucl_key).fetch1("segment_id")
    if nucl_seg_id == 0:
        raise Exception(f"No segment id for nucleus Id {nucleus_id}")
    return nucl_seg_id
    
import trimesh
def fetch_segment_id_mesh(seg_id,decimation_ratio=0.25):
    key = dict(segment_id=seg_id,decimation_ratio=decimation_ratio)
    new_mesh = (minnie.Decimation() & key).fetch1("mesh")
    current_mesh_verts,current_mesh_faces = new_mesh.vertices,new_mesh.faces
    return trimesh.Trimesh(vertices=current_mesh_verts,faces=current_mesh_faces)

def fetch_undecimated_segment_id_mesh(seg_id,decimation_ratio=0.25):
    key = dict(segment_id=seg_id,decimation_ratio=decimation_ratio)
    new_mesh = (minnie.Mesh() & key).fetch1("mesh")
    current_mesh_verts,current_mesh_faces = new_mesh.vertices,new_mesh.faces
    return trimesh.Trimesh(vertices=current_mesh_verts,faces=current_mesh_faces)


def fetch_nucleus_id_mesh(nucleus_id,verbose=False):
    seg_id = nucleus_id_to_seg_id(nucleus_id)
    print(f"Attempting to fetch segment_id: {seg_id}")
    return fetch_segment_id_mesh(seg_id)

In [6]:
n_verts,seg_ids = (segments & key_source & "n_vertices > 3000000").fetch("n_vertices","segment_id")

In [15]:
sorted_indexes = np.argsort(n_verts)
sorted_indexes

array([118, 189,  49, 119,  56,  73,   0, 190,  68,  34,  31, 168, 169,
        18, 193, 192,  82, 128, 174,  19,  74,  36,  54, 144, 143,  88,
       147,  89, 177,  84,  93,  32,  85,  64,  55, 116,  14, 138, 158,
        24, 126,  76, 164, 145, 178,  66,  58,  46,  92,  77,  12,  48,
        29, 150, 187, 170,   4,   2,   7,  97,   3,  42, 176, 173, 160,
       182, 156,  86, 100, 149,   1, 157, 132,  50, 101, 115,  78, 111,
       163,  94,  45,  11, 108,  33, 151,  75,   9,   5, 124,  69,  43,
       186,  53, 142, 107,  65,  57, 134, 165, 175,  83, 188, 159,  63,
        10, 131, 123, 161, 125,  13,  23,  62,  21, 127,  61, 162, 153,
       104,  20, 152,  40, 130,   6,  17,  35,  52, 129, 133, 148, 135,
       109, 146,  59,  95, 121,  90, 139, 106, 171,  87,  60, 180, 122,
        96,  51,  79,  15,  37, 103, 102, 185, 191, 172, 155,  44, 184,
        80, 113,  81, 167, 112,  38, 105, 166,  25,   8,  67,  70,  72,
        99,  26, 120, 137,  16, 114,  30,  47,  71, 117,  91,  2

In [9]:
np.argmax(n_verts)

41

In [10]:
seg_ids[41]

864691135579649476

In [13]:
import trimesh_utils as tu


'864691135579649476_large_axon_cloud_2.off'

In [32]:
curr_segment_id = seg_ids[sorted_indexes[-7]]
print(f"curr_segment_id = {curr_segment_id}")
new_lrg_mesh = fetch_segment_id_mesh(curr_segment_id)
nviz.plot_objects(new_lrg_mesh)

curr_segment_id = 864691135388633842


/usr/local/lib/python3.6/dist-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))


In [33]:
description = "large_upward_apical"

In [34]:
tu.write_neuron_off(new_lrg_mesh,f"{curr_segment_id}_{description}")

'864691135388633842_large_upward_apical.off'

In [8]:
from os import sys
import neuron_visualizations as nviz

# Viewing the Neurons

In [ ]:
curr_seg_id = seg_ids[7]
print(curr_seg_id)

In [ ]:
new_mesh = fetch_segment_id_mesh(curr_seg_id)
new_mesh

In [ ]:
nviz.plot_objects(meshes=new_mesh)

In [ ]:
curr_seg_id

In [ ]:
# import trimesh_utils as tu
# tu.write_neuron_off(new_mesh,"864691135548568516_single_soma_inhib_axon_cloud.off")

In [ ]:
single_soma_excitatory_large_axon = [864691135419631858]
single_soma_excitatory_large_axon_with_merges = [864691135577756100]
single_soma_inhibitory_large_axon = [864691136265012708]
double_soma_large_axon = [864691135382617330]
triple_soma_with_lots_axon = [864691136819525998]
single_soma_inhib_axon_cloud = [864691135548568516]
lots_of_error = [864691135707063188] #probably has a blood vessel